In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [ ]:
# subject_id	hadm_id patient

In [ ]:
db = sqlite3.connect('Data/ehr.sqlite')

for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    chunk = chunk.merge(df_patient, how='inner', on=['subject_id', 'hadm_id'])
    
#     chunk = chunk[(chunk['subject_id'] == ) & (chunk['hadm_id'] == )]     # filter it to the subjectId and hadm_id
    chunk.to_sql('distext', db, if_exists = 'append', index = False)            # Append the chunk to a referral table